In [107]:
pip install torch pandas numpy gensimnltk

ERROR: Could not find a version that satisfies the requirement gensimnltk (from versions: none)
ERROR: No matching distribution found for gensimnltk

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [108]:
import torch 
import re
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from gensim.models import Word2Vec
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [109]:
X = pd.read_csv('cleaned_wine_training_data.csv')
# the data is already cleaned, so we can just tokenize the words
# tokenize the description data
X['description'] = X['description'].apply(lambda x: word_tokenize(x))
# print(X['price'].isnull().values.any())
# print("BEFORE",len(X))
X.dropna(subset= ['description'], inplace=True)
X.dropna(subset= ['price'], inplace=True)
# print(X['price'].isnull().values.any())
# print("AFTER:", len(X))

# train a word2vec model on the description data
prelim = Word2Vec(X['description'], min_count=1, workers=3, window=3, sg=1)

In [110]:
# Define a custom dataset class
# Writing a custom dataset: customize data for each batch 
class CustomDataset(Dataset):
    def __init__(self, data, targets): 
        self.data = data
        self.targets = targets
        # self.hidden_layer = None TO DO ADD TO THE PARAMS or not idk 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


In [111]:
# Constructing the NN architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        #Define each layer separately
        self.fc1 = nn.Linear(input_size, hidden_size)
        # self.dropout = nn.Dropout(p = 0.2) #randomly zeros elements of the input tensor with probability p
        # self.bn = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

        
        #Define each layer sequentially 
        # self.nn = nn.Sequential(nn.Linear(input_size,hidden_size), nn.Linear(hidden_size, output_size), nn.ReLU())
    
    def forward(self, x):
        #Separate layer pass
        x = self.fc1(x)
        # x = self.dropout(x)
        # x = self.bn(x)
        x = self.relu(x)
        x = self.fc2(x)
        # self.hidden_layer = h
        return x
        # we cant return BOTH 
    
        # Sequential forward pass
        # return self.nn(x)
    
 
# Define hyperparameters DONE: WHAT ARE OUR INPUTS 
input_size = prelim.vector_size
hidden_size = 64
output_size = 1

# Set the seed for PyTorch's random number generator
torch.manual_seed(24)
np.random.seed(24)
# Initialize the model, loss function, and optimizer
model = NeuralNet(input_size, hidden_size, output_size)

In [112]:
print(model)

NeuralNet(
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)


In [113]:
model.train() # specifies that the model is being trained (tunable parameters)
# model.eval() # specifices that the model is not being trained (don't use dropout and batchnorm)

# #with torch.no_grad(): # use this to not tune model parameters 

# # model.cuda() # If you are on a gpu, this will push the model onto the gpu

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

NeuralNet(
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [120]:
#TO DO: loss = CUSTOM MSE - CONTRASTIVE LOSS # define a loss function
loss = torch.nn.MSELoss()
learning_rate = 0.001 # define a learning rate 
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # define an optimizer

In [115]:
input_data_embeddings = []
max_length = 0

for review in X['description']:
    embedding_sequence = [prelim.wv[word] for word in review if word in prelim.wv]
    input_data_embeddings.append(embedding_sequence)
    max_length = max(max_length, len(embedding_sequence))

# Pad sequences to the same length
input_data_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in input_data_embeddings]
input_data_tensors_padded = pad_sequence(input_data_tensors, batch_first=True)

# You can check the shape of input_data_tensors_padded to see the maximum length
print("Shape of padded input data:", input_data_tensors_padded.shape)

# Assuming X["price"] contains your target prices
target_tensors = torch.tensor(X["price"], dtype=torch.float32)

Shape of padded input data: torch.Size([93964, 75, 100])


In [116]:
# # Example data and actual values 
# # data = torch.randn(100, 10)  # 100 samples, 10 features
# # targets = torch.randint(0, 2, (100,))  # 100 actual values (0 or 1)
# #TO DO: GRAB COLS FROM CSVC 

# # Inputs need to be tensors
 
# # Extract Word2Vec embeddings for input data
# # Assuming 'input_data' contains your input sentences or documents

# input_data_embeddings = []
# for review in X['description']:
#     print(review, len(review))
#     # sentence_embedding = [print(prelim.wv[word]) for word in review if word in prelim.wv]
#     # for word in review:
#     #     print(prelim.wv[word])

#     # input_data_embeddings.append(sentence_embedding)

# print("len at the end", len(input_data_embeddings))

# # Convert input data embeddings to tensors
# prices = X["price"]
# input_data_tensors = torch.tensor(input_data_embeddings, dtype=torch.float32)
# target_tensors = torch.tensor(prices, dtype=torch.float32)

# # TO DO : Forward pass through the neural network
# # output = model(input_data_tensors)

# print(input_data_tensors)
# print(type(prices), prices)

# # Create custom dataset instance
# # dataset = CustomDataset(data, targets)
dataset = CustomDataset(input_data_tensors_padded, target_tensors)

In [121]:
num_epochs = 15
batch_size = 20

# Create DataLoader #TO DO: MODIFY TO TAKE IN 2 POINTS AT A TIME 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # multiprocessing supported with num_workers = n
model.train()
# Training loop
for epoch in range(num_epochs):
    for batch_data, batch_targets in dataloader:
        # print(type(batch_data), batch_data.shape)
        # print(type(batch_targets), batch_targets.shape)

        #batch_data = batch_data.cuda() # push data onto gpu
        
        # Forward pass
        outputs = model(batch_data)
        # print("this was a number before", outputs)
        temp = outputs
        # print("OUTPUTS should now be a tensor object instead of tuple", type(outputs))
        
        # calculate loss between output from model and actual values 
        pred = loss(temp, batch_targets)
        # print("PRED", pred)
        # print("outputs new :", temp)
        # print("TRUE Y", batch_targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()# zero gradient buffers
        pred.backward()# backprop
        optimizer.step()# update 
    
    # Print loss every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {pred.item():.4f}')


Epoch [1/15], Loss: 248.8989
Epoch [2/15], Loss: 202.1680
Epoch [3/15], Loss: 1358.3790
Epoch [4/15], Loss: 270.4036
Epoch [5/15], Loss: 2013.9700
Epoch [6/15], Loss: 318.1687
Epoch [7/15], Loss: 620.0620
Epoch [8/15], Loss: 248.8893
Epoch [9/15], Loss: 237.8450
Epoch [10/15], Loss: 583.8531
Epoch [11/15], Loss: 449.6649
Epoch [12/15], Loss: 235.6612
Epoch [13/15], Loss: 5863.2974
Epoch [14/15], Loss: 534.7920
Epoch [15/15], Loss: 28.7218


In [118]:
#Evaluating on test set
# model.eval()
# test_data = torch.randn(20, 10)
# test_targets = torch.randint(0, 2, (20,))
# TO DO: validation data columns 

# with torch.no_grad():
#     model_pred = model(test_data)

In [119]:
# getting indices of predictions
_, predicted = torch.max(model_pred, 1)
    
# Compare predictions with targets
correct = (predicted == test_targets).sum().item()

# Calculate accuracy
accuracy = correct / test_targets.size(0)

print("accuracy: ", accuracy)

NameError: name 'model_pred' is not defined

In [ ]:
# could also use a dataloader 
test_dataset = CustomDataset(test_data, test_targets)
dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last = False) 
test_loss = 0
correct = 0
with torch.no_grad():
    for batch_data, batch_targets in dataloader:
        batch_data, batch_targets = batch_data.to(device), batch_targets.to(device)
        pred = model(batch_data)
        test_loss += loss(pred, batch_targets).item()
        correct += (pred.argmax(1) == batch_targets).type(torch.float).sum().item()
print("test loss: ", test_loss)
print("number correct: ", correct)
print("accuracy: ", correct / test_targets.size(0))

In [ ]:
# save the model
torch.save(model.state_dict(), "448_test_nn.pth")

In [ ]:
# load the model
nn = NeuralNet(input_size, hidden_size, output_size) # need model shape/parameters
nn.load_state_dict(torch.load("448_test_nn.pth"))

In [ ]:
# save the model 
torch.save(model, '448_test_nn_full.pth')

In [ ]:
# load the model, don't need model shape or parameters
nn = torch.load('448_test_nn_full.pth')
print(nn)

In [ ]:
!pip install pipreqs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 5.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 12.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7b12a8ca2790fecb53f3e539e70407429262eb45ddf02acf887ef06c5bab9f41
  Stored in directory: /Users/ahung/Library/Caches/pip/wheels/7c/d7/8d/2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt


  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: ipython
    Found existing installation: ipython 8.7.0
    Uninstalling ipython-8.7.0:
      Successfully uninstalled ipython-8.7.0
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.5.4
    Uninstalling nbconvert-6.5.4:
      Successfully uninstalled nbconvert-6.5.4
